## Linear Model: Preprocessing Strategy ##

* Define features (X) and target variable (y).

* Split the dataset into train, test, validation sets (e.g., 70/20/10 split);

**Linear Model Preprocessor 5 steps (python object)**

*No imputation, no scaling, no capping, no encoding for price!!!*

TRAIN SET:
- Handle missing values: missingness is meaningful here; impute Nan with median and include a missing_flag indicator;
- Handle skeweness:log-transform skewed features and target, cap outliers at 1-99%;
- Encoding categorical variables (TargetEncoding and OneHotEncoder), and scaling (StandardScaling);

TEST, VALIDATION SETS:
Apply the same sub-steps from above but with the parameters learned from the **training set**.


In [2]:
import pandas as pd

filename = "cleaned_properties.csv"
df = pd.read_csv(filename)
df.columns
df

,id,price,property_type,subproperty_type,region,province,locality,zip_code,latitude,longitude,...,fl_garden,garden_sqm,fl_swimming_pool,fl_floodzone,state_building,primary_energy_consumption_sqm,epc,heating_type,fl_double_glazing,cadastral_income
0,34221000,225000.0,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,2050,51.217172,4.379982,...,0,0.0,0,0,MISSING,231.0,poor,GAS,1,922.0
1,2104000,449000.0,HOUSE,HOUSE,Flanders,East Flanders,Gent,9185,51.174944,3.845248,...,0,0.0,0,0,MISSING,221.0,poor,MISSING,1,406.0
2,34036000,335000.0,APARTMENT,APARTMENT,Brussels-Capital,Brussels,Brussels,1070,50.842043,4.334543,...,0,0.0,0,1,AS_NEW,NaN,MISSING,GAS,0,NaN
3,58496000,501000.0,HOUSE,HOUSE,Flanders,Antwerp,Turnhout,2275,51.238312,4.817192,...,0,0.0,0,1,MISSING,99.0,excellent,MISSING,0,NaN
4,48727000,982700.0,APARTMENT,DUPLEX,Wallonia,Walloon Brabant,Nivelles,1410,NaN,NaN,...,1,142.0,0,0,AS_NEW,19.0,excellent,GAS,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75503,30785000,210000.0,APARTMENT,APARTMENT,Wallonia,Hainaut,Tournai,7640,NaN,NaN,...,0,0.0,0,1,AS_NEW,NaN,MISSING,MISSING,1,NaN
75504,13524000,780000.0,APARTMENT,PENTHOUSE,Brussels-Capital,Brussels,Brussels,1200,50.840183,4.435570,...,0,0.0,0,0,AS_NEW,95.0,good,GAS,1,NaN
75505,43812000,798000.0,HOUSE,MIXED_USE_BUILDING,Brussels-Capital,Brussels,Brussels,1080,NaN,NaN,...,0,0.0,0,1,TO_RENOVATE,351.0,bad,GAS,0,NaN
75506,49707000,575000.0,HOUSE,VILLA,Flanders,West Flanders,Veurne,8670,NaN,NaN,...,1,NaN,0,1,AS_NEW,269.0,poor,GAS,1,795.0


In [3]:
#Define features (X) and target variable (y)

from sklearn.model_selection import train_test_split

X = df.drop(columns = ["price","id","zip_code","latitude","longitude"])
y = df["price"]
type(y)

pandas.core.series.Series

In [4]:
#Split the dataset into train, test, validation sets (e.g., 60/20/20 split);
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size=0.2, random_state = 86)

X_train, X_val, y_train, y_val = train_test_split(X_temp,y_temp, test_size = 0.25, random_state = 86)


In [5]:
print(type(X_train))
print(type(X_test))
print(type(X_val))
print(type(y_train))
print(type(y_test))
print(type(y_val))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Linear Model Preprocessor 5 steps ##
impute → cap → log → scale → encode

In [6]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,FunctionTransformer, StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders as ce
from category_encoders import TargetEncoder

Following the multicolinearity analysis, there are certain features candidates for dropping. Need to choose one among the identified groups



In [7]:
#NAN: replace NAN with median and add a missing_fl column here:
numeric_features = ["cadastral_income", "surface_land_sqm", "construction_year",
                    "primary_energy_consumption_sqm","nbr_bedrooms","nbr_frontages",
                    "terrace_sqm", "total_area_sqm","garden_sqm"]

skewed_features = ["surface_land_sqm","total_area_sqm", "garden_sqm","terrace_sqm"]

# No NAN to be handled, only encoding
categorical_onehot = ["property_type","region","province","heating_type","equipped_kitchen", "epc"]
categorical_target = ["subproperty_type","locality","state_building"]

# # No NAN to be handled and no encoding
binary_features = ["fl_floodzone", "fl_double_glazing", "fl_open_fire","fl_terrace", 
                        "fl_garden", "fl_swimming_pool", "fl_furnished"
                        ]


In [8]:

# Function for log-transformation
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Class Outlier Capper
class OutlierCapper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
    
    def fit(self, X, y=None):
        # Compute thresholds for each column based on training data
        self.lower_ = np.quantile(X, self.lower_quantile, axis=0)
        self.upper_ = np.quantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X):
        # Clip values to the learned thresholds
        return np.clip(X, self.lower_, self.upper_)
    
capper = OutlierCapper(lower_quantile=0.05, upper_quantile=0.95)

# Pipeline for numeric columns (imputation, scale, capping (capping needs to come as a parameter from train data - leakage issue))
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('cap',capper),
    ('scaler', StandardScaler())
])

# Pipeline for numeric features that need log-transform (specific order for cap,log,scaler)
log_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('cap', capper),
    ('log', log_transformer),
    ('scaler', StandardScaler())
])

# Pipeline for one-hot categorical features - binary features are included because "MISSING" is treated as a category
onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline for label/ordinal categorical features - "MISSING" is treated as a category
target_pipeline = Pipeline([
    ('target_enc', TargetEncoder(smoothing=1.0))
])

# Putting all pipelines together; TargetEncoder is SUPERVISED (it needs y_train)

preprocessor_linear = ColumnTransformer([
    ('num', numeric_pipeline, [f for f in numeric_features if f not in skewed_features]),
    ('log', log_pipeline, skewed_features),
    ('onehot', onehot_pipeline, categorical_onehot),
    ('label', target_pipeline, categorical_target),
    ('binary', 'passthrough', binary_features) # Just passing them as-is
])

In [9]:
# Fit on training data
X_train_processed = preprocessor_linear.fit_transform(X_train, y_train) #HERE ADDING y_train because of TargetEncoder (supervised encoder), safely handled by ColumnTransformer

# Transform test data (re-use fitted transformers)
X_test_processed = preprocessor_linear.transform(X_test)

print("X_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

# For X_train
print("First 5 rows of processed X_train:")
print(X_train_processed[:5, :5])  # first 5 rows and first 5 columns

# For X_test
print("First 5 rows of processed X_test:")
print(X_test_processed[:5, :5])

print(type(X_train_processed))
print(type(X_test_processed))

X_train_processed shape: (45304, 67)
X_test_processed shape: (15102, 67)
First 5 rows of processed X_train:
[[-1.7221717   1.18843323 -0.14593329  0.27205378 -1.27501337]
 [-0.12257671  1.26611278 -0.14593329 -1.53171094 -1.27501337]
 [-0.69457326  0.13975939 -0.86666715 -0.62982858  0.18560831]
 [-0.12257671  1.26611278 -0.14593329 -0.62982858  0.18560831]
 [-0.12257671  0.13975939 -0.14593329  0.27205378  0.18560831]]
First 5 rows of processed X_test:
[[-1.51931246  0.13975939  0.67976182  0.27205378 -1.27501337]
 [-0.12257671 -1.29731216 -0.41883252  1.17393613 -1.27501337]
 [-0.27222697  0.99423437 -1.11157672 -0.62982858 -1.27501337]
 [-0.12257671  1.26611278 -0.14593329  0.27205378  0.18560831]
 [ 0.23658391 -1.06427353  1.41449051  0.27205378 -1.27501337]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [10]:
# Get the features' names back and convert ColumnTransformer arrays into df 

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

def get_column_names(ct):
    """
    Return list of output column names produced by a fitted ColumnTransformer `ct`.
    Handles Pipelines, SimpleImputer(add_indicator=True) inside pipelines,
    and transformers that implement get_feature_names_out.
    """
    feature_names = []

    for name, transformer, cols in ct.transformers_:
        # Skip dropped transformers
        if transformer == 'drop':
            continue

        # passthrough: keep original names
        if transformer == 'passthrough':
            feature_names.extend(list(cols))
            continue

        # Some ColumnTransformer entries may be (name, transformer, slice) where
        # transformer is a Pipeline or transformer instance.
        # We'll treat Pipeline specially.
        if isinstance(transformer, Pipeline):
            
            last_step = transformer.steps[-1][1]
            if hasattr(last_step, 'get_feature_names_out'):
                try:
                    names = last_step.get_feature_names_out(cols)
                    feature_names.extend(list(names))
                    continue
                except Exception:
                    # if it fails for any reason, fall through to other checks
                    pass

            imputer_with_indicator = None
            for step_name, step_obj in transformer.steps:
                if isinstance(step_obj, SimpleImputer) and getattr(step_obj, "add_indicator", False):
                    imputer_with_indicator = step_obj
                    break

            if imputer_with_indicator is not None:
                # Imputer keeps original number of columns + indicator cols (one per input col with NaNs seen during fit)
                feature_names.extend(list(cols))
                if hasattr(imputer_with_indicator, 'indicator_'):
                    indicator_names = [f"{cols[i]}_missing_flag" for i in imputer_with_indicator.indicator_.features_]
                    feature_names.extend(indicator_names)
                continue

            feature_names.extend(list(cols))
            continue

        # If transformer is not a Pipeline
        # Try to use get_feature_names_out if present
        if hasattr(transformer, 'get_feature_names_out'):
            try:
                names = transformer.get_feature_names_out(cols)
                feature_names.extend(list(names))
                continue
            except Exception:
                pass

        # Check if this transformer itself is a SimpleImputer with add_indicator=True
        if isinstance(transformer, SimpleImputer) and getattr(transformer, "add_indicator", False):
            feature_names.extend(list(cols))
            if hasattr(transformer, 'indicator_'): # Thie priece resolves the issue when missing_fl colummn is created but not needed, causing issue when converting to df
                indicator_names = [f"{cols[i]}_missing_flag" for i in transformer.indicator_.features_]
                feature_names.extend(indicator_names)
            continue

        # final fallback: original column names
        feature_names.extend(list(cols))

    return feature_names

column_names = get_column_names(preprocessor_linear)

print("len(column_names) =", len(column_names))
print("X_train_processed.shape[1] =", X_train_processed.shape[1])
print(column_names)

# If they match, convert to DataFrame
if len(column_names) == X_train_processed.shape[1]:
    X_train_processed_df = pd.DataFrame(X_train_processed, columns=column_names)
    X_test_processed_df  = pd.DataFrame(X_test_processed,  columns=column_names)
else:
    raise ValueError(f"Column count mismatch: names={len(column_names)} vs array columns={X_train_processed.shape[1]}")


len(column_names) = 67
X_train_processed.shape[1] = 67
['cadastral_income', 'construction_year', 'primary_energy_consumption_sqm', 'nbr_bedrooms', 'nbr_frontages', 'cadastral_income_missing_flag', 'construction_year_missing_flag', 'primary_energy_consumption_sqm_missing_flag', 'nbr_frontages_missing_flag', 'surface_land_sqm', 'total_area_sqm', 'garden_sqm', 'terrace_sqm', 'surface_land_sqm_missing_flag', 'total_area_sqm_missing_flag', 'garden_sqm_missing_flag', 'terrace_sqm_missing_flag', 'property_type_APARTMENT', 'property_type_HOUSE', 'region_Brussels-Capital', 'region_Flanders', 'region_Wallonia', 'region_missing', 'province_Antwerp', 'province_Brussels', 'province_East Flanders', 'province_Flemish Brabant', 'province_Hainaut', 'province_Limburg', 'province_Liège', 'province_Luxembourg', 'province_Namur', 'province_Walloon Brabant', 'province_West Flanders', 'province_missing', 'heating_type_CARBON', 'heating_type_ELECTRIC', 'heating_type_FUELOIL', 'heating_type_GAS', 'heating_type

In [11]:
# Testing the output per pipeline block and whether the number of columns match the names
column_names = get_column_names(preprocessor_linear)
print("Transformer blocks and cols:")
for name, transformer, cols in preprocessor_linear.transformers_:
    print("BLOCK:", name, "cols:", cols)
    print("  is pipeline:", isinstance(transformer, Pipeline))
    if isinstance(transformer, Pipeline):
        print("  pipeline steps:", transformer.named_steps.keys())
        for step_name, step in transformer.steps:
            print("    step:", step_name, type(step), getattr(step, "add_indicator", False), hasattr(step, "get_feature_names_out"))
    else:
        print("  transformer type:", type(transformer), hasattr(transformer, "get_feature_names_out"))


onehot_cols = preprocessor_linear.named_transformers_['onehot'].named_steps['encoder'].get_feature_names_out(categorical_onehot)
print("OneHotEncoder output cols:", len(onehot_cols))

Transformer blocks and cols:
BLOCK: num cols: ['cadastral_income', 'construction_year', 'primary_energy_consumption_sqm', 'nbr_bedrooms', 'nbr_frontages']
  is pipeline: True
  pipeline steps: dict_keys(['imputer', 'cap', 'scaler'])
    step: imputer <class 'sklearn.impute._base.SimpleImputer'> True True
    step: cap <class '__main__.OutlierCapper'> False False
    step: scaler <class 'sklearn.preprocessing._data.StandardScaler'> False True
BLOCK: log cols: ['surface_land_sqm', 'total_area_sqm', 'garden_sqm', 'terrace_sqm']
  is pipeline: True
  pipeline steps: dict_keys(['imputer', 'cap', 'log', 'scaler'])
    step: imputer <class 'sklearn.impute._base.SimpleImputer'> True True
    step: cap <class '__main__.OutlierCapper'> False False
    step: log <class 'sklearn.preprocessing._function_transformer.FunctionTransformer'> False False
    step: scaler <class 'sklearn.preprocessing._data.StandardScaler'> False True
BLOCK: onehot cols: ['property_type', 'region', 'province', 'heating_ty

In [12]:
print(type(y_train))

<class 'pandas.core.series.Series'>


In [13]:
# (in)sanity check

X_train['fl_swimming_pool'].value_counts()
y_train.groupby(X_train['fl_swimming_pool']).mean()

fl_swimming_pool
0    4.112319e+05
1    1.040320e+06
Name: price, dtype: float64

In [14]:
# Log-transform PRICE
 
y_train_log = np.log1p(y_train)
y_test_log  = np.log1p(y_test) 

## Training and Testing the Linear Model ##

In [15]:
# Training the Linear model

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_processed_df, y_train_log)

# Coefficients with feature names
coef_df = pd.DataFrame({
    'feature': X_train_processed_df.columns,
    'coefficient': model.coef_
}).sort_values(by='coefficient', key=abs, ascending=False)

print(coef_df.head(25))  # top 10 strongest features

# Example interpretation below: increasing total area by 1 std oncreased log(price) by 0.23, corresponding
# to around 26% increase in price.  


                                feature  coefficient
10                       total_area_sqm     0.231797
65                     fl_swimming_pool     0.220506
35                  heating_type_CARBON    -0.141772
63                           fl_terrace    -0.137180
48  equipped_kitchen_USA_HYPER_EQUIPPED     0.126990
27                     province_Hainaut    -0.106291
46       equipped_kitchen_NOT_INSTALLED    -0.105935
12                          terrace_sqm     0.093480
21                      region_Wallonia    -0.092023
43      equipped_kitchen_HYPER_EQUIPPED     0.090743
47       equipped_kitchen_SEMI_EQUIPPED    -0.088347
53                              epc_bad    -0.084062
3                          nbr_bedrooms     0.083469
0                      cadastral_income     0.078632
54                        epc_excellent     0.077334
56                             epc_poor    -0.076411
41                   heating_type_SOLAR     0.069558
42                    heating_type_WOOD    -0.

In [16]:
# NOT NECESSARY
# Checking significance (not possible with sklearn)
#import statsmodels.api as sm

#X = X_train_processed_df
#y = y_train_log

# Add intercept term
#X_sm = sm.add_constant(X)

#ols_model = sm.OLS(y, X_sm).fit()
#print(ols_model.summary())

In [ ]:
# Testing the model

y_train_pred_log = model.predict(X_train_processed_df) # First level check for underfitting (training error too high)
y_test_pred_log = model.predict(X_test_processed_df) # Prediction on the unseen data
y_test_pred = np.expm1(y_test_pred_log) 
y_train_pred = np.expm1(y_train_pred_log)

train_results = pd.DataFrame({
    "actual train": y_train,
    "predicted train": np.expm1(model.predict(X_train_processed_df))
})

test_results = pd.DataFrame({
    "actual test": y_test,
    "predicted test": np.expm1(model.predict(X_test_processed_df))
})

print(train_results.head())
print(test_results.head())

       actual train  predicted train
19103      470453.0    304466.425110
59696      215000.0    206506.618299
73754      229000.0    254941.871304
12840      324138.0    277699.333738
11058      499000.0    477718.971121
       actual test  predicted test
40080     155000.0   144239.230691
43868     585000.0   469521.124976
56179     201000.0   202858.685318
48114     595398.0   591107.418504
3304      295000.0   309561.631443


In [18]:
# Back-transform coefficients for interpretation

## Metrics regression ##

In [19]:
# Adjusted R-sqr, RMSE

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2_log = r2_score(y_test_pred_log, np.log1p(y_test))
mse_log = mean_squared_error(np.log1p(y_test), y_test_pred_log)

r2 = r2_score(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred)

print("R2 (original scale):", r2)
print("RMSE:", rmse)
print("R2 (log scale):", r2_log)
print("RMSE (log scale):", np.sqrt(mse_log))


R2 (original scale): 0.33954826665613613
RMSE: 143477781261.8997
R2 (log scale): 0.4645265851207946
RMSE (log scale): 0.3419016547868019


In [20]:
# Over/under-fitting test
# train < test - overfitting (model memorizes training data)
# train > test - underfitting (model is too simple or wrong features)
#

def evaluate(y_true, y_pred): # temporarily defining here, not globally
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred)
    }

train_metrics = evaluate(y_train, y_train_pred)
test_metrics  = evaluate(y_test,  y_test_pred)

print("TRAIN:", train_metrics)
print("TEST: ", test_metrics)

# Result interpretation: NO over/under-fitting  

TRAIN: {'MAE': 122895.95500355208, 'RMSE': np.float64(332277.1726435241), 'R2': 0.40316051400669195}
TEST:  {'MAE': 124093.27323195233, 'RMSE': np.float64(378784.6106455484), 'R2': 0.33954826665613613}


## Full Pipeline ##

In [ ]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor_linear),
    ('model', LinearRegression())
])

full_pipeline.fit(X_train, y_train)

y_pred = full_pipeline.predict(X_test)